In [2]:
import nltk
import requests
import warnings
from sqlalchemy import text
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer

import sqlite3

# Data manipulation and analysis
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sqlalchemy.orm import sessionmaker
import matplotlib.dates as mdates

from datetime import datetime
from datetime import timedelta

# Financial data extraction
import yfinance as yf

# SQL Alchemy for database interaction
from sqlalchemy import create_engine

# Additional libraries for analysis or visualization
import matplotlib.pyplot as plt
import seaborn as sns
from config import api_key

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetches historical stock prices for the specified ticker within the given timeframe.
    """
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

def fetch_all_stock_data(tickers, start_date, end_date):
    stock_data = {}
    for ticker in tickers:
        print(f"Fetching data for {ticker}")
        stock_data[ticker] = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

# Function to fetch headlines from NewsAPI
def fetch_news(api_key, ticker):
    base_url = "https://newsapi.org/v2/everything"
    params = {
        'q': ticker,             # Search query (ticker symbol)
        'sortBy': 'publishedAt', # Sort by publication date
        'apiKey': api_key        # Your NewsAPI key
    }
    response = requests.get(base_url, params=params)
    articles = response.json().get('articles', [])
    headlines = [article['title'] for article in articles]
    return headlines

def load_data_to_db(data, ticker):
    """
    Loads transformed data into the SQLite database.
    """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    data.to_sql(ticker, con=engine, if_exists='replace', index=True)

def get_avg_sentiment_scores():
    textual_data = {}
    tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']

    for ticker in tickers:
        headlines = fetch_news(api_key, ticker)
        textual_data[ticker] = headlines

    # Analyzing sentiment for each headline and averaging the scores
    average_sentiment_scores = {}

    for ticker, headlines in textual_data.items():
        sentiments = [TextBlob(headline).sentiment.polarity if headline else 0 for headline in headlines]
        average_sentiment_scores[ticker] = np.mean(sentiments)

    # Extract tickers and sentiment scores
    tickers = list(average_sentiment_scores.keys())
    sentiment_scores = list(average_sentiment_scores.values())

    df = pd.DataFrame(sentiment_scores, tickers,['Average Sentiment Score'])
    
    load_data_to_db(df, 'Average_Sentiment_Score')


def calculate_daily_return(stock_data):
    """
    Calculates the daily return from the closing prices.
    """
    stock_data['Daily_Return'] = stock_data['Close'].pct_change()
    return stock_data

def read_data(ticker):
    """
    Reads the data for the specified ticker from the SQLite database.
    """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    # engine = create_engine('sqlite:///stock_market_analysis.db')
    query = f"SELECT * FROM '{ticker}'"
    data = pd.read_sql_query(query, con=engine)
    return data

def plot_moving_averages(stock_data, ticker):
    data = stock_data[ticker]
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    
    plt.figure(figsize=(14, 7))
    plt.plot(data['Close'], label='Close Price', alpha=0.5)
    plt.plot(data['SMA_50'], label='50-Day SMA')
    plt.plot(data['SMA_200'], label='200-Day SMA')
    plt.title(f"{ticker} - Moving Averages")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.show()

def predict_monthly_prices(model, last_known_price, last_known_date, months=36):
    future_prices = []
    future_dates = []
    current_price = last_known_price
    
    for month in range(months):
        # Approximating each month by 21 trading days
        for day in range(21):
            next_input = pd.DataFrame(data=[[current_price]], columns=['Previous Close'])
            current_price = model.predict(next_input)[0]
        
        future_prices.append(current_price)
        # Assuming last_known_date is a datetime object; add roughly 30 days for each month
        last_known_date += timedelta(days=30)
        future_dates.append(last_known_date)
    
    return future_dates, future_prices

def train_model_for_ticker(ticker_data):
    """
    Trains a LinearRegression model for the given ticker data.
    """
    data = ticker_data[['Close']].copy()
    data['Previous Close'] = data['Close'].shift(1)
    data.dropna(inplace=True)  # Drop the first row which now contains NaN

    X = data[['Previous Close']]  # Features
    y = data['Close']  # Target

    model = LinearRegression()
    model.fit(X, y)  # Training the model
    return model

# Objective Function (Negative Sharpe Ratio)
def neg_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate=0.01):
    p_var = np.dot(weights.T, np.dot(cov_matrix, weights))
    p_ret = np.dot(weights, expected_returns)
    return -(p_ret - risk_free_rate) / np.sqrt(p_var)

def analyze_sentiment(text):
    """Analyze the sentiment of a text and return polarity and subjectivity."""
    analysis = TextBlob(text)
    return analysis.sentiment.polarity, analysis.sentiment.subjectivity

def main():
    tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']

    for ticker in tickers:
        print(f"Fetching news for: {ticker}")
        articles = fetch_news(api_key, ticker)
        if articles['status'] == 'ok':
            for article in articles['articles'][:5]:  # Show only the first 5 articles for brevity
                description = article.get('description') or article.get('title')
                polarity, subjectivity = analyze_sentiment(description)
                print(f"Ticker: {ticker}, Title: {article['title']}")
                print(f"Sentiment Polarity: {polarity:.2f}, Sentiment Subjectivity: {subjectivity:.2f}")
                print(f"URL: {article['url']}\n")
    
    return articles

def fetch_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="5y")
    return data['Close']

def calculate_parameters(tickers):
    prices = {ticker: fetch_data(ticker) for ticker in tickers}
    prices = pd.DataFrame(prices)
    daily_returns = prices.pct_change()
    annual_mean_returns = daily_returns.mean() * 252
    annual_std_devs = daily_returns.std() * np.sqrt(252)
    return {ticker: {'mean': annual_mean_returns[ticker], 'std': annual_std_devs[ticker]}
            for ticker in tickers}

def calculate_expected_returns(tickers):
    prices = pd.DataFrame({ticker: fetch_data(ticker) for ticker in tickers})
    daily_returns = prices.pct_change()
    average_daily_returns = daily_returns.mean()
    annual_expected_returns = average_daily_returns * 252  # Convert to annual returns
    return annual_expected_returns



In [3]:
engine = create_engine('sqlite:///stock_market_analysis.db')

In [4]:
from sqlalchemy.orm import Session

In [5]:
session = Session(engine)

In [6]:
session.execute('SELECT * FROM Average_Sentiment_Score').all()

OperationalError: (sqlite3.OperationalError) no such table: Average_Sentiment_Score
[SQL: SELECT * FROM Average_Sentiment_Score]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
con = sqlite3.connect(engine)

In [ ]:
    # engine = create_engine('sqlite:///stock_market_analysis.db')
    query = f"SELECT * FROM '{ticker}'"
    data = pd.read_sql_query(query, con=engine)
    return data

In [7]:
read_data('Average_Sentiment_Score').to_json()

'{"index":{"0":"MSFT","1":"CRM","2":"CRWD","3":"ZM","4":"SHOP","5":"AAPL"},"Average Sentiment Score":{"0":0.0375813131,"1":0.033,"2":0.0918553391,"3":0.0210360195,"4":0.1574544913,"5":0.0192785354}}'

In [8]:
if request.args.get('api_key') and request.args['api_key'] == '4497341d87444fddb894bbc281c9f9f8':
    return f(*args, **kwargs)
else:
    return jsonify({"error": "Authentication required"}), 403


SyntaxError: 'return' outside function (828057532.py, line 2)

In [9]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    # Save to CSV
    stock_data.to_csv(f'{ticker}_stock_data.csv')
    return stock_data

  

In [10]:
def fetch_all_stock_data(tickers, start_date, end_date):
    stock_data = {}
    for ticker in tickers:
        print(f"Fetching data for {ticker}")
        data = yf.download(ticker, start=start_date, end=end_date)
        data.to_csv(f'{ticker}_all_stock_data.csv')
        stock_data[ticker] = data
    return stock_data


In [11]:
# Here is an improved version of the fetch_all_stock_data function with error handling:

def fetch_all_stock_data(tickers, start_date, end_date):
    stock_data = {}
    for ticker in tickers:
        try:
            print(f"Fetching data for {ticker}")
            data = yf.download(ticker, start=start_date, end=end_date)
            if not data.empty:
                data.to_csv(f'{ticker}_all_stock_data.csv')
                stock_data[ticker] = data
            else:
                print(f"No data found for {ticker}")
        except Exception as e:
            print(f"Failed to fetch data for {ticker}: {e}")
    return stock_data

In [12]:
## Extending the Existing Function; First, let's add a function to load the data into a SQLite database 
## (or any other relational database you prefer):

from sqlalchemy import create_engine

def load_data_to_db(data, ticker):
    engine = create_engine('sqlite:///stock_data.db')  # Change to your database URL
    data.to_sql(ticker, con=engine, index=False, if_exists='replace')
    print(f"Data for {ticker} loaded into database")

## Flask API to Serve Data; To create an API that serves this data, you'll implement a Flask application. 
## Here's a basic example:
from flask import Flask, jsonify, request
from sqlalchemy import create_engine
import pandas as pd

app = Flask(__name__)

def get_data_from_db(ticker):
    engine = create_engine('sqlite:///stock_data.db')  # Ensure this matches the DB used in load_data_to_db
    data = pd.read_sql(ticker, con=engine)
    return data

@app.route('/stocks/<ticker>', methods=['GET'])
def serve_stock_data(ticker):
    try:
        data = get_data_from_db(ticker)
        return jsonify(data.to_dict(orient='records'))
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "/Users/yakupaltinisik/anaconda3/envs/dev/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/yakupaltinisik/anaconda3/envs/dev/lib/python3.11/site-packages/traitlets/config/application.py", line 991, in launch_instance
    app.initialize(argv)
  File "/Users/yakupaltinisik/anaconda3/envs/dev/lib/python3.11/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yakupaltinisik/ana

SystemExit: 1

/Users/yakupaltinisik/anaconda3/envs/dev/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [42]:
# First, we need to ensure that your fetch_news function is well integrated and can fetch news headlines 
# as required. We'll also write the sentiment analysis into the same DataFrame and save it:

import requests
import pandas as pd
from textblob import TextBlob
from sqlalchemy import create_engine

api_key = 'YOUR_API_KEY'

def fetch_news(api_key, ticker):
    base_url = "https://newsapi.org/v2/everything"
    params = {
        'q': ticker,             # Search query (ticker symbol)
        'sortBy': 'publishedAt', # Sort by publication date
        'apiKey': api_key        # Your NewsAPI key
    }
    response = requests.get(base_url, params=params)
    articles = response.json().get('articles', [])
    headlines = [article['title'] for article in articles]
    return headlines

def analyze_sentiment(headlines):
    """ Return DataFrame with headlines and their sentiment scores """
    sentiments = [TextBlob(headline).sentiment.polarity for headline in headlines]
    df = pd.DataFrame({
        'Headline': headlines,
        'Sentiment': sentiments
    })
    return df

def get_avg_sentiment_scores():
    tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']
    all_data = pd.DataFrame()

    for ticker in tickers:
        headlines = fetch_news(api_key, ticker)
        df = analyze_sentiment(headlines)
        df['Ticker'] = ticker
        all_data = pd.concat([all_data, df])

    all_data.to_csv('sentiment_scores.csv', index=False)
    return all_data


## Load Data to Database Your existing load_data_to_db function looks fine, but make sure you have 
## the appropriate database setup and SQLAlchemy installed:

def load_data_to_db(data, ticker):
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    data.to_sql(ticker, con=engine, if_exists='replace', index=True)
    
    
## Flask API to Serve Data as JSON. Now, let’s set up a Flask API to serve the saved data:    

from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/api/sentiment/<ticker>', methods=['GET'])
def get_sentiment(ticker):
    try:
        engine = create_engine('sqlite:///stock_market_analysis.sqlite')
        query = f"SELECT * FROM '{ticker}'"
        data = pd.read_sql_query(query, con=engine)
        return jsonify(data.to_dict(orient='records'))
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=True)

# Execute the Python script containing the get_avg_sentiment_scores function to fetch the data, 
# analyze sentiment, and save it to both a CSV file and a SQLite database.

 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
On macOS, try disabling the 'AirPlay Receiver' service from System Preferences -> Sharing.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [44]:
#Optimizing and Integrating the Function
#Here's an optimized version of your function with additional comments and improvements 
#for clarity and functionality:

import numpy as np
import pandas as pd
from textblob import TextBlob
import requests

def fetch_news(api_key, ticker):
    """Fetches news headlines for a given ticker using NewsAPI."""
    base_url = "https://newsapi.org/v2/everything"
    params = {
        'q': ticker,             # Search query (ticker symbol)
        'sortBy': 'publishedAt', # Sort by publication date
        'apiKey': api_key        # Your NewsAPI key
    }
    response = requests.get(base_url, params=params)
    articles = response.json().get('articles', [])
    headlines = [article['title'] for article in articles]
    return headlines

def get_avg_sentiment_scores(api_key):
    """Calculates average sentiment scores from news headlines for specified tickers."""
    tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']
    textual_data = {ticker: fetch_news(api_key, ticker) for ticker in tickers}

    # Compute average sentiment scores
    average_sentiment_scores = {}
    for ticker, headlines in textual_data.items():
        sentiments = [TextBlob(headline).sentiment.polarity for headline in headlines if headline]
        average_score = np.mean(sentiments) if sentiments else 0
        average_sentiment_scores[ticker] = average_score

    # Create DataFrame and save to CSV
    df = pd.DataFrame(list(average_sentiment_scores.items()), columns=['Ticker', 'Average Sentiment Score'])
    df.to_csv('average_sentiment_scores.csv', index=False)
    
    return df

### Integrating with Flask API: To make these sentiment scores available via your Flask API, 
### you can add an endpoint that executes this function and returns the results in JSON format:
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/sentiment_scores', methods=['GET'])
def sentiment_scores():
    api_key = request.args.get('api_key')
    if not api_key:
        return jsonify({"error": "API key is required"}), 401
    try:
        df = get_avg_sentiment_scores(api_key)
        return jsonify(df.to_dict(orient='records'))
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
On macOS, try disabling the 'AirPlay Receiver' service from System Preferences -> Sharing.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [45]:
def load_data_to_db(data, ticker):
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    # Save transformed data before loading to DB
    data.to_csv(f'{ticker}_transformed_data.csv')
    data.to_sql(ticker, con=engine, if_exists='replace', index=True)
    


In [46]:
## Expanding on Your Function; You provided a function that loads data into a SQLite database and 
## also saves it as a CSV file for backup or further use:

from sqlalchemy import create_engine
import pandas as pd

def load_data_to_db(data, ticker):
    """
    Saves DataFrame to CSV and loads it into a SQLite database.

    Args:
    data (DataFrame): The data to be stored.
    ticker (str): The table name in the database.

    """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    # Save transformed data before loading to DB
    data.to_csv(f'{ticker}_transformed_data.csv')
    data.to_sql(ticker, con=engine, if_exists='replace', index=True)
    
## Setting Up Flask API to Serve Data: To read from the database and serve data via a Flask API, 
## you would need additional functionalities:

from flask import Flask, jsonify
from sqlalchemy import create_engine
import pandas as pd

app = Flask(__name__)

def get_data_from_db(ticker):
    """ Retrieve data from the database for a given ticker. """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    query = f"SELECT * FROM '{ticker}'"
    data = pd.read_sql_query(query, con=engine)
    return data

@app.route('/data/<ticker>', methods=['GET'])
def serve_data(ticker):
    """
    Serve data for the given ticker.
    """
    try:
        data = get_data_from_db(ticker)
        return jsonify(data.to_dict(orient='records'))
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
On macOS, try disabling the 'AirPlay Receiver' service from System Preferences -> Sharing.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [47]:
def plot_moving_averages(stock_data, ticker):
    data = stock_data[ticker]
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()

    # Save to CSV
    data.to_csv(f'{ticker}_moving_averages.csv')

    plt.figure(figsize=(14, 7))
    plt.plot(data['Close'], label='Close Price', alpha=0.5)
    plt.plot(data['SMA_50'], label='50-Day SMA')
    plt.plot(data['SMA_200'], label='200-Day SMA')
    plt.title(f"{ticker} - Moving Averages")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.show()


In [48]:
## Modifications to the Function; Here’s an enhanced version of your function with added error handling and 
## efficiency improvements:
import pandas as pd
import matplotlib.pyplot as plt

def plot_moving_averages(stock_data, ticker):
    try:
        data = stock_data.get(ticker)
        if data is None:
            raise ValueError("Ticker data is not available")
        
        # Calculate Simple Moving Averages
        data['SMA_50'] = data['Close'].rolling(window=50).mean()
        data['SMA_200'] = data['Close'].rolling(window=200).mean()

        # Save to CSV
        data.to_csv(f'{ticker}_moving_averages.csv')

        # Plotting
        plt.figure(figsize=(14, 7))
        plt.plot(data['Close'], label='Close Price', alpha=0.5)
        plt.plot(data['SMA_50'], label='50-Day SMA')
        plt.plot(data['SMA_200'], label='200-Day SMA')
        plt.title(f"{ticker} - Moving Averages")
        plt.xlabel("Date")
        plt.ylabel("Price (USD)")
        plt.legend()
        plt.show()
        
    except Exception as e:
        print(f"Error processing data for {ticker}: {e}")

### Integrating with Flask API; Here’s how you can integrate this functionality into a Flask API setup:

from flask import Flask, jsonify, request
from sqlalchemy import create_engine
import pandas as pd

app = Flask(__name__)

def get_stock_data_from_db(ticker):
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    query = f"SELECT * FROM '{ticker}'"
    data = pd.read_sql_query(query, con=engine)
    return data

@app.route('/moving_averages/<ticker>', methods=['GET'])
def moving_averages(ticker):
    try:
        stock_data = get_stock_data_from_db(ticker)
        plot_moving_averages({ticker: stock_data}, ticker)  # Assuming plot function does not return but saves and shows plot
        return jsonify(stock_data[['Date', 'SMA_50', 'SMA_200']].to_dict(orient='records'))
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
On macOS, try disabling the 'AirPlay Receiver' service from System Preferences -> Sharing.


AttributeError: 'tuple' object has no attribute 'tb_frame'